# Tugas Besar IF 3270 Pembelajaran Mesin
Dikumpulkan: Jumat, 5 Maret 2021 jam 23.55

Dibuat oleh
Kelompok **Neuron Activation**
* Farras Mohammad Hibban Faddila - K2 - 13518017
* Jun Ho Choi Hedyatmo - K2 - 13518044
* Moch. Nafkhan Alzamzami - K3 - 13518132
* Michel Fang - K2 - 13518137

## Bagian A: Implementasi Forward Propagation untuk Feed Forward Neural Network

In [140]:
import numpy as np

## Fungsi Aktivasi

Fungsi aktivasi yang dikenali adalah linear, sigmoid, ReLU, dan softmax.


In [141]:
def sigmoid(x):
    # applying the sigmoid function
    return 1 / (1 + np.exp(-x))

In [142]:
def sigmoid_derivative(x):
    # computing derivative to the Sigmoid function
    return x * (1 - x)

In [143]:
def relu(x):
    # compute relu
    return np.maximum(0, x)

In [144]:
def relu_derivative(x):
    return np.where(x <= 0, 0, 1)

In [145]:
def linear(x):
    return x

In [146]:
def linear_derivative(x):
    return np.full(x.shape, 1)

In [147]:
def softmax(x):
    ex = np.exp(x)
    return ex / np.sum(ex)

## Pemilihan Fungsi Aktivasi

In [148]:
activation_functions = {
    # activation_functions
    "relu": relu,
    "sigmoid": sigmoid,
    "softmax": softmax,
    "linear": linear
}

## Class Layer

Setiap layer dibatasi memiliki neuron dengan fungsi aktivasi yang sama, namun antar layer diperbolehkan memiliki neuron dengan aktivasi yang berbeda.

In [149]:
class Layer:
    def __init__(self, activation, input, output):
        self.activation_name = activation
        self.activation = activation_functions[activation]
        self.W = np.zeros((output, input))
        self.b = np.zeros((output, 1))

    def set_weight(self, weight):
        self.W = weight

    def set_bias(self, bias):
        self.b = bias

## Class NeuralNetwork
Konvensi nama method dibuat mirip dengan library machine learning pada umumnya.

In [150]:
class NeuralNetwork:
    def __init__(self):
        # seeding for random number generation
        np.random.seed(1)
        # converting weights to a 3 by 1 matrix with values from -1 to 1 and mean of 0
        self.layers = []
        self.depth = 0

    def add(self, layer):
        self.layers.append(layer)
        self.depth += 1

### Membaca dan menyimpan model FFNN
Input dan output adalah sebuah file yang nama filenya `filename`.

Format data model adalah
```
<depth>
<n_neuron>
(for every neuron)
<activation_type>
a11 a12 a13 (weights)
a21 a22 a23
b1 b2 (bias)
```

Contoh:
```
1
2
sigmoid
a11 a12 a13
a21 a22 a23
b1 b2
```

In [151]:
class NeuralNetwork(NeuralNetwork):
    def load_model(self, filename):
        f = open(filename, "r")

        self.depth = int(f.readline())

        for i in range(self.depth):

            n_neuron = int(f.readline())
            activation_type = f.readline()[:-1]
            weight = []

            n_neuron_prev = -1
            for j in range(n_neuron):
                temp = list(map(float, f.readline().split()))
                weight.append(temp)
                if (n_neuron_prev == -1):
                    n_neuron_prev = len(temp)

            layer = Layer(activation_type, n_neuron_prev, n_neuron)
            layer.set_weight(np.array(weight))
            bias = np.array(
                list(map(lambda x: [float(x)], f.readline().split())))
            layer.set_bias(bias)

            self.layers.append(layer)

    def save_model(self, filename):
        f = open(filename, "w")

        f.write(str(self.depth) + "\n")
        for layer in self.layers:
            n_neuron = len(layer.b)
            f.write(str(n_neuron) + "\n")
            f.write(layer.activation_name + "\n")
            for i in range(n_neuron):
                f.write(" ".join(list(map(str, layer.W[i]))) + "\n")
            f.write(
                " ".join(list(map(lambda x: str(x[0]), layer.b))) + "\n")

### Forward propagation

In [152]:
class NeuralNetwork(NeuralNetwork):
    def forward_propagate(self, x_inputs):
        a = np.array(x_inputs).T
        for layer in self.layers:
            z = np.dot(layer.W, a) + layer.b
            a = layer.activation(z)
        return a

    def backward_propagate(self):
        pass

### Training dan prediksi

In [153]:
class NeuralNetwork(NeuralNetwork):
    def fit(self, x_train, y_train, training_iterations):
        # training the model to make accurate predictions while adjusting weights continually
        for iteration in range(training_iterations):
            # siphon the training data via the neuron
            prediction = self.forward_propagate(x_train)
            # cost_function = error(prediction, y_train)
            self.backward_propagate()

    def predict(self, x_test):
        prediction = self.forward_propagate(x_test)
        return prediction

### Menampilkan model dengan struktur dan koefisien

In [154]:
class NeuralNetwork(NeuralNetwork):
    def __str__(self):
        index = 1
        res = ""
        for layer in self.layers:
            res += "{}-th layer\n".format(index)
            res += f"Activation: {layer.activation_name}\n"
            res += "Weight matrix:\n"
            res += layer.W.__str__() + "\n"
            res += "Bias\n"
            res += layer.b.__str__() + "\n\n"
            index += 1
        return res

### Data XOR
Data berikut diambil dari slide kuliah

In [155]:
DATA = [[0, 0], [0, 1], [1, 0], [1, 1]]

### Model XOR Sigmoid

In [156]:
xor_model = NeuralNetwork()
xor_model.load_model("xor-sigmoid.txt")
print(xor_model)

1-th layer
Activation: sigmoid
Weight matrix:
[[ 20.  20.]
 [-20. -20.]]
Bias
[[-10.]
 [ 30.]]

2-th layer
Activation: sigmoid
Weight matrix:
[[20. 20.]]
Bias
[[-30.]]




Prediksi Sigmoid

In [157]:
xor_model.predict(DATA)

array([[4.54391049e-05, 9.99954520e-01, 9.99954520e-01, 4.54391049e-05]])

### Model XOR ReLU & Linear

In [158]:
relu_linear_model = NeuralNetwork()
relu_linear_model.load_model("xor-relu-linear.txt")
print(relu_linear_model)

1-th layer
Activation: relu
Weight matrix:
[[1. 1.]
 [1. 1.]]
Bias
[[ 0.]
 [-1.]]

2-th layer
Activation: linear
Weight matrix:
[[ 1. -2.]]
Bias
[[0.]]




Prediksi ReLU & Linear

In [159]:
relu_linear_model.predict(DATA)

array([[0., 1., 1., 0.]])